### Funktionen zum Spiegeln

Wie zuvor erklärt, werden in diesem Notebook Stellungen gespiegelt, um die Effizienz der Berechnung zu erhöhen.
Der folgende Abschnitt erklärt die damit verbundende Funktionalität.

Die globale Variable `g_swaps` stellt für jede Spiegelungsart ein Dictionary zur Verfügung.
Diese Dictionaries besitzen als `key` und `value` ein `chess.Square`.
Der `key` steht dabei für die ursprüngliche Position und der `value` für die gespiegelte Position.

Um die bitweisen Operationen für die Spiegelungen nachvollziehen zu können, gilt es sich zunächst die Nummerierung der Felder zu betrachten:
![Nummerierung Schachfelder](Images/square_numbering.png)

Bei der binären Betrachtung der Schachfelder kann man speziell in den Ecken ein Muster erkennen:
* $(0)_{10} = (000000)_2$
* $(7)_{10} = (000111)_2$
* $(56)_{10} = (111000)_2$
* $(63)_{10} = (111111)_2$

Anhand dieser Darstellungen haben sich speziell für die horizontale und vertikale Spiegelungen folgende binäre Operationen gefunden.
> Horizontale Spiegelung: mirrored = square \^ 56
> Vertikale Spiegelung: mirrored = square \^ 7

Diese Formeln haben sich daraus ergeben, dass die 3 niederwertigen Bit für die Position der Spalte und die 3 höherwertigen Bits für die Position der Zeile verwendet werden.

Aus dieser Erkenntnis konnten auch die Spiegelungen an den Diagonalen gefolgert werden.
Die Spiegelung an der aufsteigenden Diagonalen (0-9-18-27-36-45-54-63) hat zur Folge, dass die Indizes der Zeile und Spalte miteinander vertauscht werden.
Die binäre Operation hierfür lautet:
> Spiegelung an aufsteigenden Diagonalen: mirrored = ((square >> 3) | (square << 3)) & 63

Bei dieser Formel stellt der Teil `& 63` sicher, dass die nötigen Bits zur Beschreibung des Felds nicht überschritten werden.

Für das Spiegeln an der fallenden Diagonalen (56-49-42-35-28-21-14-07) kann das Spiegeln an der aufsteigenden Diagonalen genutzt werden.
Zu dieser Spiegelung kommt noch die Spiegelung anhand der vertikalen und horizontalen Achse (Umsetzung mit `^ 63`) hinzu.
Die Kombination dieser Operationen führt zur Spiegelung an der abfallenden Diagonalen. Dementsprechend lautet die Formel:
> Spiegelung an abfallenden Diagonalen: mirrored = (((square >> 3) | (square << 3)) & 63) ^ 63

Anhand dieser zuvor beschriebenen Spiegelmethoden ist es zusätzlich noch möglich die Rotationen um 90°, 180°, 270° darzustellen, da diese ebenfalls durch mehrfache Spiegelungen umgesetzt werden können.
Beispielsweise ist die Drehung um 180° eine Spiegelung sowohl an der vertikalen, als auch horizontalen Ebene.
Die restlichen Operationen können aus der Variablen `g_swaps` entnommen werden.

In [ ]:
g_swaps = {
        "vertical" : {x : (x ^ 56) for x in range(64)},
        "horizontal" : {x : (x ^ 7) for x in range(64)},
        "rotate_right" : {x : ((((x >> 3) | (x << 3)) & 63) ^ 56) for x in range(64)},
        "rotate_180" : {x : (x ^ 63) for x in range(64)},
        "rotate_left" : {x: ((((x >> 3) | (x << 3)) & 63) ^ 7) for x in range(64)},
        "diagonal" : {x: (((x >> 3) | (x << 3)) & 63) for x in range(64)},
        "anti_diagonal" : {x: ((((x >> 3) | (x << 3)) & 63) ^ 63) for x in range(64)}
    }

Die Spiegelung wird von der Funktion `mirror_board(board, swap)` für eine Stellung `board` durchgeführt.
Hierzu wird ein neues `chess.Board`-Objekt erstellt.
Für dieses werden mithilfe des übergebenen Dictionaries `swap` die neuen Positionen der vorhandenen Schachfiguren bestimmt und gesetzt.
Das gespiegelte Schachbrett wird von der Funktion zurückgegeben.

In [ ]:
def mirror_board(board, swap):
    swapped_board = chess.Board(None)
    swapped_board.turn = board.turn

    for position, piece in board.piece_map().items():
        swapped_board.set_piece_at(swap[position], piece)
    return swapped_board


Die Funktion `mirror_all_directions(board_int, piece_list)` erstellt für einen Integer (`board_int`) eine Menge aller Spiegelungen mit den Dictionaries, die in `g_swaps` aufgeführt sind und der Funktion `mirror_board(board, swap)`.
Da für die Funktion `mirror_board(board, swap)` ein `chess.Board` benötigt wird, wird zusätzlich eine `piece_list` zur Erstellung des Objekts mitgegeben.
Die Menge `result` enthält den originalen Integer `board_int` nicht.

In [ ]:
def mirror_all_directions(board_int, piece_list):
    result = set()
    board = to_board(board_int, piece_list)
    for name, swap in g_swaps.items():
        result.add(to_integer(mirror_board(board, swap), piece_list))
    return result


Die Funktion `get_all_mirror_variations(uniques_int, piece_list)` berechnet für eine Menge von Stellungen in Integer-Repräsentation (`uniques_int`) die Menge der gespiegelten Variationen inklusive der originalen Stellungen (`result`).
Hierfür wird die Funktion `mirror_all_directions(unique, piece_list)` verwendet, weshalb auch eine `piece_list` übergeben wird.
Die Variable `result` wird von der Funktion `get_all_mirror_variations(uniques_int, piece_list)` zurückgegeben.

In [ ]:
def get_all_mirror_variations(uniques_int, piece_list):
    result = uniques_int.copy()
    for unique in uniques_int:
        result |= mirror_all_directions(unique, piece_list)
    return result


Soll eine zuvor berechnete Endspielsituation zur Auswertung verwendet werden, müssen alle $n$ gespiegelt werden.
Die Erstellung der vollständigen $S_n$ Mengen geschieht durch die Funktion `gen_all_integers(s_n_integers, piece_list)`.
Diese nimmt die bereits gespeicherten Mengen in ``s_n_integers`` und iteriert über diese.
Für jedes Brett der Menge werden die 7 Spiegelungen unter Verwendung der `piece_list` generiert.
Diese werden in einer Liste von Mengen (`result`) gespeichert.
Die Indizes der Mengen werden dabei nicht verändert. Die Liste `result` stellt den Rückgabewert der Funktion dar.

In [ ]:
def gen_all_integers(s_n_integers, piece_list):
    result = []
    i = 0
    n = len(s_n_integers)
    for sequence in s_n_integers:
        int_set = set()
        for int_board in sequence:
            int_set.add(int_board)
            int_set |= mirror_all_directions(int_board, piece_list)
        result.append(int_set)
        print("Done mirroring Set " + str(i) + " of " + str(n))
        i += 1
    return result
